# Homework 6

In [1]:
import numpy as np 
import random
import math

# Get the inputs ready
 
i = 0
trainX = np.zeros((1000, 10))
trainY = np.zeros ((1000, 1))
testX = np.zeros((1000, 10))
testY = np.zeros ((1000, 1))
input1 = np.zeros((1000, 11))
input2 = np.zeros((1000, 11))

with open ("train.txt") as trainingData :
    for line in trainingData:
        input1[i] = line.split(" ")        
        i = i + 1

i = 0
with open ("test.txt") as testData :
    for line in testData:
        input2[i] = line.split(" ")
        #print(input[i])
        i = i + 1

for i in range (0, 1000):
    for j in range (0, 11):
        if (j == 10):
            trainY[i][0] = input1[i][j]
            testY[i][0] = input2[i][j]
        else:
            trainX[i][j] = input1[i][j]
            testX[i][j] = input2[i][j]

Split Data  
Using Decision Stump to determine the best (i, $\theta$)  
That gives the purest data   
add two impurity weighted by their size

In [7]:
# Compute Impurity
def Gini (data, theta, s):
    size = len(data)
    if (size == 0.0):
        return 1.0
    length = len(data[0])
    gini = 0.0
    gini2 = 0.0
    # calculate number of 1s
    # size - gini = number of zeros
    for i in data :
        if (i[length-1] == 1):
            gini = gini + 1
        elif (i[length-1] == -1):
            gini2 = gini2 + 1
    gini = gini / size
    gini2 = gini2 / size
    gini = gini**2
    gini2 = gini2**2
    return (1-gini-gini2)

def SplitData (data):
    impurity = np.zeros((1000, 10))
    besti = 0
    bestj = 0
    bestTheta = 0.0
    theta = 0.0
    s = 0
    bests=0
    purest = 1
    size = len(data)
    if (size == 0):
        return [], [], -1, 0, 1
    for i in range (len(data[0])-1):
        if (i == 10):
            break
        arr = sorted(data, key = lambda x:x[i])
        for j in range (len(arr)):
            if (j == 0):
                theta = -100
                continue
            elif (j != 0):
                theta = (arr[j-1][i] + arr[j][i])/2
                #print (j-1, j, theta)
            
            #for sig in [1, -1]:
            left, right = list(), list()
            lsize, rsize = 0, 0
            for row in arr:                 
                if ((row[i] - theta) < 0):
                    left.append (row)
                    lsize = lsize + 1
                else :
                    right.append(row)
                    rsize = rsize + 1
            impurity[j][i] = (lsize*Gini (left, theta, 1) + rsize*Gini (right, theta, 1))/(lsize+rsize)
            '''if (sig == 1):
                p1=(lsize*Gini (left, theta, 1) + rsize*Gini (right, theta, 1))/(lsize+rsize)
            else :
                p2 = (lsize*Gini (left, theta, -1) + rsize*Gini (right, theta, -1))/(lsize+rsize)
        if (p1 < p2):
            impurity[j][i] = p1
            s = 1
        else :
            impurity[j][i] = p2
            s = -1'''
            if (impurity[j][i] < purest):
                purest = impurity[j][i]
                besti = i
                bestj = j
                bestTheta = theta
                bests = 1
    left, right = list(), list()
    length = len(data[0])-1
    err = 0
    for k in data :
        if (k[length] * bests * (k[besti]-bestTheta) < 0):
            err = err + 1
    if (err/len(data) > 0.5):
        bests = -1
    for row in data :
        if (bests * (row[besti]-bestTheta) < 0):
            left.append(row)
        else :
            right.append(row)

    print (f"Splitted with value {theta:.4f}, j {bestj:.4f}, i {besti:.4f}, purest {purest:.4f}")
    print (len(left), len(right))
    if (bestj == 0):
        return [], [], -1, 0, 1
    return left, right, bestTheta, besti, bests


Now we have to build the tree  
and stop the tree from growing when we reach base case  



In [8]:
class node :
    def __init__ (self, data=[], value = -1, parent = None, Lchild = None , Rchild = None, index = 0, s=1) :
        self.parent = parent
        self.data = data 
        self.value = value
    
    def pure (self) :       
        if (self.data == [] or len(self.data) == 0):
            return True
        size = len(self.data[0])-1
        temp = self.data[0][len(self.data[0])-1]
        for row in self.data : 
            if (row[size] != temp):
                return False
        return True
        #np.all(self.data[:][len(self.data[0])-1]==temp)

    def __getitem__ (self, index):
        return self.data[index]


count = 0
def split_tree (root, count):
    count = count + 1
    print (count)
    if (root.pure() == True):
        return 
    else:
        ldata, rdata, theta, index, s  = SplitData(root.data)
        root.value = theta
        root.index = index
        root.s = s
        root.Lchild = node(parent=root, data=ldata, value=theta)
        root.Rchild = node(parent=root, data=rdata, value=theta)
        if (not root.Lchild.pure()):
            split_tree (root.Lchild, count)
        if (not root.Rchild.pure()):
            split_tree (root.Rchild, count)
    return

root = node (data = input1)
split_tree (root, count)



1
Splitted with value 2.6710, j 100.0000, i 5.0000, purest 0.4444
100 900
2
Splitted with value 2.6710, j 784.0000, i 9.0000, purest 0.4401
116 784
3
Splitted with value 2.6710, j 97.0000, i 2.0000, purest 0.1787
97 19
4
Splitted with value 2.6710, j 15.0000, i 4.0000, purest 0.1007
82 15
5
Splitted with value 2.5729, j 27.0000, i 0.0000, purest 0.0452
55 27
6
Splitted with value 1.9728, j 26.0000, i 0.0000, purest 0.0712
26 1
7
Splitted with value 1.9728, j 4.0000, i 2.0000, purest 0.0577
22 4
8
Splitted with value 1.2874, j 3.0000, i 0.0000, purest 0.0000
3 1
5
Splitted with value 2.2381, j 10.0000, i 7.0000, purest 0.1067
10 5
6
Splitted with value 2.0810, j 4.0000, i 1.0000, purest 0.0000
1 4
4
Splitted with value 1.9798, j 7.0000, i 0.0000, purest 0.2368
7 12
5
Splitted with value 1.7362, j 9.0000, i 5.0000, purest 0.0000
3 9
3
Splitted with value 0.9691, j 84.0000, i 9.0000, purest 0.4202
84 700
4
Splitted with value -0.9187, j 4.0000, i 0.0000, purest 0.1902
80 4
5
Splitted with

predict with the tree we just generated

In [9]:
def predict (root, test_data):
    if (root.pure()):
        i = root.parent.index
        return ( root.parent.s * np.sign(test_data[i]-root.parent.value))
    threshold = root.value 
    i = root.index
    if (test_data[i] < threshold):
        return predict (root.Lchild, test_data)
    else:
        return predict (root.Rchild, test_data)

predicted_value = np.zeros((1000, 1))
for i in range (len(input2)) :
    predicted_value[i][0] = predict (root, input2[i])

error = 0.0
for i in range (len(input2) ) :
    if (predicted_value[i][0] * input2[i][len(input2[0])-1] == -1):
        error = error + 1

print (error / len(input2))



0.469


In [20]:
for L in range (1, 51):
    arr = np.zeros((L))
    

maxi = 0
for i in range (1, 50):
    d1 = i
    d2 = 50 - i
    if (20*(d1-1) + d1*(d2-1) + d2*3 > maxi):
        maxi = 20*(d1-1) + d1*(d2-1) + d2*3
print (maxi)


maxi = 0
for i in range (1, 49):
    for j in range (1, 49-i):
        d1 = i
        d2 = j
        d3 = 50-i-j
        if (20*(d1-1) + d1*(d2-1) + d2*(d3-1) + d3*3 > maxi):
            maxi = 20*(d1-1) + d1*(d2-1) + d2*(d3-1) + d3*3
print (maxi)


1219
1123


In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

df = pd.read_csv('train.csv')
print(df.info())
#print(df.loc[1,:])
#print(df.head())

df_train = df.drop(columns = ['is_canceled', 'adr', 'reservation_status', 'reservation_status_date'])
df_hidden = df[['is_canceled', 'adr', 'reservation_status', 'reservation_status_date']]
print(df_train.info())
print(df_hidden.info())

df_test = pd.read_csv('test.csv')
print(df_test.info())

'''
df_wc = pd.concat([df_train, df_hidden])
print(df_wc.info())
df_wr = pd.concat([df_train, df_test])
print(df_wr.info())
'''

df_cat = pd.concat([df_train, df_test], ignore_index = True)
print(df_cat.info())

print(df_cat.iloc[0:91531, :].reset_index(drop=True))
print(df_cat.iloc[91531:119390, :].reset_index(drop=True))

print(df_cat)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
df_encoded = pd.get_dummies(df_cat, columns = ["hotel", "arrival_date_month", "meal", "country", "market_segment", 
                                         "distribution_channel", "reserved_room_type", "assigned_room_type", 
                                         "deposit_type", "customer_type", "agent", "company"], dummy_na = True)

print(df_encoded.info())

idx = df_encoded[df_encoded['children'].isnull()].index.tolist()
print(idx)
df_encoded["children"][idx] = 0
print(df_encoded["children"][idx])
        
df_encoded = df_encoded.dropna()

print('#########################')
print(df_cat.info())
print(df_cat.shape)
print('#########################')
print(df_encoded.info())
print(df_encoded.shape)
print('#########################')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


regr = RandomForestRegressor (bootstrap=True, oob_score=True, criterion = "mae")

regr.fit (df_x, df_label)
label_pred = regr.predict(df_x)

regr.score(df_x, df_adr)